# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files(r'data/song_data')

In [5]:
filepath = song_files[0] # First file only
print(filepath)

/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json


In [6]:
df = pd.read_json(filepath, typ='series')
df.head()
list(df.values)[0]

1

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = []
song_data.append(df['song_id'])
song_data.append(df['title'])
song_data.append(df['artist_id'])
song_data.append(df['year'])
song_data.append(df['duration'])
song_data


['SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, 209.60608]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = []
artist_data.append(df['artist_id'])
artist_data.append(df['artist_name'])
artist_data.append(df['artist_location'])
artist_data.append(df['artist_latitude'])
artist_data.append(df['artist_longitude'])
artist_data

['ARXR32B1187FB57099', 'Gob', '', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [4]:
log_files = get_files('data/log_data')

In [5]:
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-13-events.json'

In [6]:
df = pd.read_json(filepath, lines=True)
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Kevin,M,0,Arellano,NaN,free,"Harrisburg-Carlisle, PA",GET,Home,1.540007e+12,514,None,200,1542069417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,None,Logged In,Maia,F,0,Burke,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540677e+12,510,None,200,1542071524796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df['datetime'] = pd.to_datetime(df['ts'], unit='ms')
filtered = df[df['page'] == 'NextSong']

time_data = [
    filtered.datetime
    ,filtered.datetime.dt.hour
    ,filtered.datetime.dt.day
    ,filtered.datetime.dt.weekofyear
    ,filtered.datetime.dt.month
    ,filtered.datetime.dt.year
    ,filtered.datetime.dt.weekday
]

labels = ['timestamp', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday']

ts_dict = dict(zip(labels, time_data))
time_df = pd.DataFrame.from_dict(ts_dict)
time_df.head()

,timestamp,hour,day,weekofyear,month,year,weekday
1,2018-11-13 00:40:37.796,0,13,46,11,2018,1
3,2018-11-13 01:12:29.796,1,13,46,11,2018,1
4,2018-11-13 03:19:02.796,3,13,46,11,2018,1
5,2018-11-13 03:51:52.796,3,13,46,11,2018,1
8,2018-11-13 05:00:06.796,5,13,46,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [15]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [7]:
# Getting user data frame. All NULL users are being set to user_id = 0
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
#user_df.loc[df['userId'] == '', 'userId'] = 0
user_df.head(5)

,userId,firstName,lastName,gender,level
0,66,Kevin,Arellano,M,free
1,66,Kevin,Arellano,M,free
2,51,Maia,Burke,F,free
3,51,Maia,Burke,F,free
4,9,Wyatt,Scott,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [8]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

DataError: invalid input syntax for integer: ""
LINE 3: VALUES ('', NULL, NULL, NULL, 'free')
                ^


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [18]:
h = df.head(5)
h.loc[h['song'] == 'Ja I Ty', 'song'] = 'Face the Ashes'
h.loc[h['artist'] == 'Fu', 'artist'] = 'Gob'
h.loc[h['length'] == 280.05832, 'length'] = 209.60608
for index, row in h.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    if(row.song == 'Face the Ashes'):
        print(row.song, row.artist, row.length)
    #print(results)
    if results:
        songid, artistid = results
        songplay_data = (0, str(row.datetime), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent.replace('\"', ''))
        print(songplay_data)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
    else:
        songid, artistid = None, None
        
    #if(songid is not None):
    #print(songid)
        
    # insert songplay record
    #songplay_data = (0, row.ts, int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent.replace('\"', ''))
    #print(songplay_data)
    #cur.execute(songplay_table_insert, songplay_data)
    #conn.commit()

#songplay_id int
#,start_time int
#,user_id int
#,level varchar
#,song_id varchar
#,artist_id varchar
#,session_id int
#,location varchar
#,user_agent_id varchar

Face the Ashes Gob 209.60608
(0, '2018-11-13 00:40:37.796000', '66', 'free', 'SOFSOCN12A8C143F5D', 'ARXR32B1187FB57099', 514, 'Harrisburg-Carlisle, PA', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36')


/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.